# Black Litterman Model

In [37]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [38]:
# Definindo o range de datas... Período de 5 anos
start_date = '2006-01-01'
end_date = '2018-12-31'

symbols = ['ABEV3','BBAS3','BBDC3','BRFS3','CCRO3','GGBR4','ITSA4',
           'ITUB4','LREN3','PETR3','PETR4','VALE3']

dates = pd.date_range(start_date, end_date)

dfBase = pd.DataFrame(index = dates)

dfCDI = pd.read_excel("C:/Users/Viviane/Desktop/Data/CDI.xlsx", index_col = "Date", parse_dates = True,na_values=['nan'])
dfCDI = pd.DataFrame(dfCDI['CDI'])

dfIBOV = pd.read_csv("C:/Users/Viviane/Desktop/Data/IBOV.csv", index_col = "Date", parse_dates = True,
                     usecols = ['Date', 'Adj Close'], na_values=['nan'])

dfIBOV = dfIBOV.dropna()
dfIBOV = dfIBOV.rename(columns={'Adj Close': 'IBOV'})

dfBase = dfBase.join(dfCDI['CDI'], how = 'inner')
dfBase = dfBase.join(dfIBOV, how = 'inner')

for symbol in symbols:
    df_temp = pd.read_csv("C:/Users/Viviane/Desktop/Data/{}.csv".format(symbol), index_col = "Date", parse_dates = True, 
                     usecols = ['Date', 'Adj Close'], na_values=['nan'])
    df_temp = df_temp.rename(columns={'Adj Close': symbol})
    dfBase = dfBase.join(df_temp)

dfBase.fillna(method = "ffill", inplace = True)
dfBase.fillna(method = "bfill", inplace = True)

In [39]:
daily_Return = dfBase.copy()
daily_Return = np.log(daily_Return / daily_Return.shift(1))
daily_Return.iloc[0,:] = 0

In [40]:
Excess_Mkt_Return = np.array(daily_Return['IBOV']) - np.array(daily_Return['CDI'])
price_of_risk = np.mean(Excess_Mkt_Return) / np.var(daily_Return['IBOV'])
price_of_risk

2.035118996066036

In [41]:
port_daily_Return = daily_Return.drop('IBOV',axis = 1).drop('CDI',axis = 1)
Covariance_matrix = port_daily_Return.cov()
Covariance_matrix

,ABEV3,BBAS3,BBDC3,BRFS3,CCRO3,GGBR4,ITSA4,ITUB4,LREN3,PETR3,PETR4,VALE3
ABEV3,0.000320,0.000164,0.000152,0.000122,0.000124,0.000157,0.000144,0.000160,0.000159,0.000147,0.000143,0.000164
BBAS3,0.000164,0.000759,0.000388,0.000223,0.000281,0.000359,0.000405,0.000433,0.000321,0.000423,0.000425,0.000324
BBDC3,0.000152,0.000388,0.000461,0.000182,0.000220,0.000292,0.000357,0.000382,0.000258,0.000330,0.000320,0.000270
BRFS3,0.000122,0.000223,0.000182,0.000539,0.000159,0.000201,0.000211,0.000206,0.000205,0.000198,0.000180,0.000191
CCRO3,0.000124,0.000281,0.000220,0.000159,0.000566,0.000206,0.000218,0.000225,0.000215,0.000231,0.000245,0.000188
GGBR4,0.000157,0.000359,0.000292,0.000201,0.000206,0.000807,0.000325,0.000325,0.000273,0.000421,0.000426,0.000497
ITSA4,0.000144,0.000405,0.000357,0.000211,0.000218,0.000325,0.000982,0.000444,0.000289,0.000327,0.000344,0.000281
ITUB4,0.000160,0.000433,0.000382,0.000206,0.000225,0.000325,0.000444,0.000512,0.000296,0.000358,0.000343,0.000309
LREN3,0.000159,0.000321,0.000258,0.000205,0.000215,0.000273,0.000289,0.000296,0.000696,0.000279,0.000259,0.000263
PETR3,0.000147,0.000423,0.000330,0.000198,0.000231,0.000421,0.000327,0.000358,0.000279,0.001450,0.000747,0.000452


In [42]:
mkt_weights = [278.562, 137.299, 261.886, 18.158, 27.209, 23.69, 116.46, 333.782, 32.34, 350.253, 350.272, 270.673]
weights = []
for i in range(len(mkt_weights)):
    weights.append(mkt_weights[i]/sum(mkt_weights))
weights

[0.12658548821585544,
 0.062392074103965134,
 0.11900749982731859,
 0.008251445979794455,
 0.012364445074580203,
 0.010765324113962476,
 0.05292231516724651,
 0.15167882707499464,
 0.014696098853758824,
 0.15916365837432245,
 0.15917229244600525,
 0.12300053076819609]

In [43]:
Implied_Equilibrium_Exc_Returns = np.dot(Covariance_matrix,weights)*price_of_risk
Implied_Equilibrium_Exc_Returns

array([0.00035337, 0.00079418, 0.00065724, 0.00038441, 0.00044294,
       0.00072542, 0.00073048, 0.00072389, 0.00054272, 0.001169  ,
       0.00094139, 0.00076532])

In [44]:
tau = 1 / len(daily_Return) #Fator tau (Incerteza das estimativas)
                            #gerado de acordo com Meucci, sendo o inverso do tamanho da amostra.
tau

0.0003114294612270321

In [45]:
views = np.array([0.0 for i in range(1,37)]).reshape((12,3))

#Petro (absolute view)
views[9,0] = 0.5
views[10,0] = 0.5

#BBAS perfomaria mais que ITAU (relative view)
views[1,1] = 1.0
views[6,1] = -0.5
views[7,1] = -0.5

#Vale sobre Gerdau (relative view)
views[11,2] = 1.0
views[5,2] = -1.0

views

array([[ 0. ,  0. ,  0. ],
       [ 0. ,  1. ,  0. ],
       [ 0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ],
       [ 0. ,  0. , -1. ],
       [ 0. , -0.5,  0. ],
       [ 0. , -0.5,  0. ],
       [ 0. ,  0. ,  0. ],
       [ 0.5,  0. ,  0. ],
       [ 0.5,  0. ,  0. ],
       [ 0. ,  0. ,  1. ]])

In [48]:
Q_matrix = np.array([0.0525,0.03,0.01])
confidence = np.array([0.25,0.50,0.65])

In [53]:
k = views.shape[0]
omega = np.zeros((k,k))

for i in range(k):
    for j in range(k):
        if (i == j):
            omega = np.dot(np.dot((tau * views), Covariance_matrix),view)

ValueError: shapes (12,3) and (12,12) not aligned: 3 (dim 1) != 12 (dim 0)